# Data Scraping
### By: Adam Aharony, adam.aharony@gmail.com, 214435448.

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from time import sleep
import pandas as pd
import random


def wait():
    r = random.uniform(3, 10)
    sleep(r)


In [ ]:
option = webdriver.ChromeOptions()

option.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36")
# option.add_argument("--profile-directory=Default")
# option.add_argument("--user-data-dir=C:/Users/adama/AppData/Local/Google/Chrome/User Data")
option.add_argument('--disable-blink-features=AutomationControlled')
option.add_experimental_option("useAutomationExtension", False)
option.add_experimental_option("excludeSwitches", ["enable-automation"])

# Open Browser
driver = webdriver.Chrome(executable_path='chromedriver.exe', options=option)
driver.execute_script(
    "Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

site = "https://www.techpowerup.com/gpu-specs/?mfgr={manufacturer}"
manufacturers = ("3dfx", "AMD", "ATI", "Intel",
                 "Matrox", "NVIDIA", "Sony", "XGI")


In [ ]:
ROWS = []


In [ ]:
# for manufacturer in manufacturers:
manufacturer = manufacturers[7]
driver.get(site.format(manufacturer=manufacturer))
# Getting released years
years_select = Select(driver.find_element(By.ID, "released"))
years = [year.text for year in years_select.options][1:]
years = [y[:y.index(" ")] for y in years]
wait()
for year in years:
    driver.get(site.format(manufacturer=manufacturer) + f"&released={year}")
    elems = driver.find_elements(By.CLASS_NAME, f"vendor-{manufacturer}")
    names = [e.find_element(By.TAG_NAME, "a").text for e in elems]
    links = [e.find_element(By.TAG_NAME, "a") for e in elems]
    addresses = [link.get_attribute("href") for link in links]
    for name, address in zip(names, addresses):
        row = {"Name": name, "Link": address, "Year": year}
        ROWS.append(row)

    wait()


In [ ]:
df = pd.DataFrame(ROWS)
df


In [ ]:
df.to_csv(f"{manufacturer}.csv")


In [ ]:
manufacturer = manufacturers[7]


In [ ]:
ROWS = []


In [ ]:
df = pd.read_csv(f"manufacturers/{manufacturer}.csv")
# df = df.drop(columns=["Unnamed: 0"])
for i, row in df.iterrows():
    ROWS.append(dict(row))
ROWS


In [ ]:
for i, row in enumerate(ROWS):
    if list(row.keys()) == ["Name", "Link", "Year"]:
        driver.get(row["Link"])
        wait()
        celems = driver.find_elements(By.CLASS_NAME, "gpudb-specs-large__title")
        relems = driver.find_elements(By.CLASS_NAME, "gpudb-specs-large__value")
        ROWS[i].update({c.text: r.text for c, r in zip(celems, relems)})
        clearfix = driver.find_elements(By.CLASS_NAME, "clearfix")
        for c in clearfix:
            celems = c.find_elements(By.TAG_NAME, "dt")
            relems = c.find_elements(By.TAG_NAME, "dd")
            ROWS[i].update({c.text: r.text for c, r in zip(celems, relems)})
        wait()


In [ ]:
df = pd.DataFrame(ROWS)
df


In [ ]:
ROWS[-1]


In [ ]:
# save list:
import pickle
with open("ROWS.pickle", "wb") as f:
    pickle.dump(ROWS, f)


In [ ]:
# read list:
import pickle
with open("ROWS.pickle", "rb") as f:
    ROWS = pickle.load(f)


In [ ]:
df.to_csv(f"manufacturers/{manufacturer}.csv", index=False)


In [ ]:
df.drop(columns=["Unnamed: 0"], inplace=True)
